In [ ]:
# This will clone and install the branch that has the current changes on it. If you want to see the output of the install to be sure it 
# was successfull, remove the > /dev/null 2>&1 in the second line.  

#!pip install -e astrohack/ > /dev/null 2>&1
!pip install -e .. > /dev/null 2>&1

In [ ]:
import os
import json
import astrohack

import numpy as np

In [ ]:
import distributed

from menrva.client import local_client

DEFAULT_DASK_ADDRESS="tcp://localhost:8786"

log_params = {
    'logger_name': "astrohack",
    'log_level':'INFO',
    'log_to_term':True,
    'log_to_file': False,
}

if not distributed.client._get_global_client():
    try:
        client = distributed.Client(DEFAULT_DASK_ADDRESS, timeout=2)
        
    except OSError:
        print("Spawn client ...")
        os.environ["DASK_SCHEDULER_ADDRESS"] = DEFAULT_DASK_ADDRESS
        client=local_client(
            cores=2,
            memory_limit='8GB',
            log_params=log_params
        )
        
else:
    client = distributed.client._get_global_client()


In [ ]:
client.dashboard_link

In [ ]:
astrohack.data.datasets.download('ea25_cal_small_before_fixed.split.ms', folder="data")

ms_file = "data/ea25_cal_small_before_fixed.split.ms"

In [ ]:
from astrohack.extract_pointing import extract_pointing

point_mds = extract_pointing(
    ms_name=ms_file,
    point_name="data/ea25_cal_small_before_fixed.split.point.zarr",
    #exclude=["ea25"],
    parallel=True,
    overwrite=True
)

In [ ]:
# Here I reduced the size of the holog_obs_dict for quicker testing. You can comment this out of edit it as needed depending on your testing goals.

from astrohack.extract_holog import generate_holog_obs_dict

holog_obs_dict = generate_holog_obs_dict(
    ms_name=ms_file,
    point_name="data/ea25_cal_small_before_fixed.split.point.zarr",
)

### The holog observations dictionary has now been turned into an extended dictionary object similar to the `holog_mds`. There are a couple of new functionalities available to the user now:

- The `select` function allows the user to trim the holog_obs_dict on all major axes (ddi, map, antenna, scan, baseline) in a much simpler way than previsouly.
- The `print` function allows the user to inspect the holog_obs_dict with both a static, ascii print out style and a dynamic, collapsible json style.
- A static method is available to allow the user to select the `n` closest baseline associated with a given mapping antenna adn return them in a list that cane be used with the baseline key option allowing the user to trim the holog_obs_dict based on baseline.

### Each of these options is demonstrated below.

In [ ]:
#get_nearest_baselines(antenna="ea25", n_baselines=5)

In [ ]:
# There are two options here for style, static and dynamic. Static returns a formatted ascii output while dynamic returns a dynamic json object.
holog_obs_dict.print(style="dynamic")

In [ ]:
from astrohack.extract_holog import model_memory_usage

memory_per_core = model_memory_usage(
    ms_name=ms_file, 
    holog_obs_dict=holog_obs_dict
)

### `Select(key, value, inplace, **kwargs)`
### The select function allows the user to trim the `holog_obs_dict`. The `key` (ddi, map, antenna, scan, baseline) input gives the dictionary key you would like to select and `value` is the value you want to keep. Everything else is deleted. There is also an `inplace` boolean input. This specifies whether you want to modify the calling dictionay or return a new dictionary. The additional arguments, `kwarags` is currently only useful for the baseline example explain further down.

### One key feature is also that all select calls are chainable. This allows for selecting on mutiple keys in a simple modular way. An example is shown below.

In [ ]:
# Select only ddi=0 and return a new dictionary, from the original dictionary above.
#trimmed_dict = holog_obs_dict.select(key="ddi", value=1, inplace=False)
#trimmed_dict.print(style="dynamic")

In [ ]:
# Here we will select only following (key,value) pairs: 
#
# ddi=0
# map=0
# scans=6, 7, 8, 9, 10
# antenna='ea22', 'ea15', 'ea25'
#
# We do this using function chaining. Because each call returns a new object, another call can be directly invoked.

#trimmed_dict = holog_obs_dict.select(
#    key="ddi", 
#    value=0, 
#    inplace=False
#).select(
#    key="map", 
#    value=0, 
#    inplace=False
#).select(
#    key="scan", 
#    value=[6, 7, 8, 9, 10], 
#    inplace=False
#).select(
#    key="antenna", 
#    value=['ea22', 'ea15', 'ea25'], 
#    inplace=False
#)

#trimmed_dict.print(style="dynamic")

### `Select(...)` with the baseline option:

### In order to trim the dictionary based on the shortest baseline formed using a given mapping antenna we require the use of two functions. First we use the static method to get the reference antennas that for the `n` shortest baselines given and antenna.

In [ ]:
# Since it doesn't mdoify the dictionary object, this can be called on the new returned dictionary or the original dictionary. The following 
# gives the antennas for the three shortest baselines formed with the mapping antenna, 'ea25'. Note these don't take into account whether the 
# antenna is reference or mapping, it only calculates the baseline lengths for reference.
#nearest_baselines = holog_obs_dict.get_nearest_baselines(antenna="ea15", n_baselines=24)
#nearest_baselines
#
#.select(
#    key="scan", 
#    value=[
#        6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
#        16, 17, 19, 20, 21, 22, 23, 24, 25, 
#        26, 27, 28, 29, 30, 35, 36, 37, 38, 
#        39, 40
#    ],
#    inplace=False
#)

In [ ]:
# With the reference antenna list in-hand we can now trim the dictionary using the select method as follows
'''
trimmed_dict = {}

trimmed_dict = holog_obs_dict.select(
    key="ddi", 
    value=1,
    inplace=False
).select(
    key="baseline", 
    value="ea15", 
    n_baselines=3, 
    inplace=False
)

trimmed_dict.print(style="dynamic")
'''

In [ ]:
# You will notice that there is a warning thrown below regarding the mutliple values of cell size and pixel number found in the pointing file
# but suggested values are computed. They are NOT guarenteed to work though. For instance, below you will see the number of pixels is very small
# and if you use the resonably calculated pixle value suggested, you will get a singular matrix. There is still work to be done.

from astrohack.extract_holog import extract_holog

holog_mds = extract_holog(
    ms_name=ms_file,
    point_name="data/ea25_cal_small_before_fixed.split.point.zarr",
    #holog_name="data/otf33.holog.zarr",
    #holog_obs_dict=trimmed_dict,
    data_column='CORRECTED_DATA',
    parallel=True,
    overwrite=True
)

In [ ]:
from astrohack.holog import holog

image_mds = holog(
    holog_name="data/ea25_cal_small_before_fixed.split.holog.zarr",
    #grid_size=np.array([30, 30]),
    overwrite=True,
    parallel=True
)

In [ ]:
#
# A list of colormaps available in matplotlib can be found here:
#
# https://matplotlib.org/stable/users/explain/colors/colormaps.html
#

image_mds.plot_apertures(
    destination="plots", 
    ant="ea06",
    colormap="RdYlGn",
    display=True
)

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

panel_mds = panel(
    image_name='data/ea25_cal_small_before_fixed.split.image.zarr', 
    panel_model=panel_model, 
    panel_margins=0.2,
    clip_type='relative',
    clip_level=0.2,
    parallel=True,
    overwrite=True
)

In [ ]:
panel_mds.plot_antennas(
    "panel_exports",         # Directory to contain the plot and text file
    ant='ea06',              # Plotting Antenna ea06
    ddi=0,                   # Plotting DDI 0
    plot_type='deviation',   # Do deviation plots only
    deviation_unit='mils',   #
    plot_screws=False,       # Not plotting screw positions
    parallel=False,          # Don't do plots in parallel
    display=True             # Display plots below
)

In [ ]:
client.shutdown()